In [2]:
import json
import openai
import dotenv

In [3]:
dotenv.load_dotenv()

client = openai.Client()

In [4]:
def greeting_by_period(hour):
    if 5 <= 5 < 12:
        return json.dumps({"greeting": "Good morning"})
    elif 12 <= hour < 18:
        return json.dumps({"greeting": "Good afternoon"})
    elif 18 <= hour < 22:
        return json.dumps({"greeting": "Good afternoon"})
    else:
        return json.dumps({"greeting": "Good night"})

In [15]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "greeting_by_period",
            "description": "Return greeting by day hour",
            "parameters": {
                "type": "object",
                "properties": {
                    "hour": {
                        "type": "integer",
                        "description": "Day hour in twenty four hours"
                    },
                },
                "required": ["hour"]
            }
        }
    }
]

available_functions = {
    "greeting_by_period": greeting_by_period,
}

In [52]:
messages = [{"role": "user", "content": "What greeting does the model give me if it is 23 o'clock"}]

In [53]:
answer = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

answer_message = answer.choices[0].message
tool_calls = answer_message.tool_calls

In [54]:
answer_message

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_GibCoLe8whSruqNTtyu91Oeb', function=Function(arguments='{"hour":23}', name='greeting_by_period'), type='function')])

In [55]:
tool_calls

[ChatCompletionMessageToolCall(id='call_GibCoLe8whSruqNTtyu91Oeb', function=Function(arguments='{"hour":23}', name='greeting_by_period'), type='function')]

In [56]:
if tool_calls:
    messages.append(answer_message)
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(hour=function_args.get("hour"))

        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": function_name,
            "content": function_response
        }
        )

    second_answer = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
    )

In [57]:
second_answer.choices[0].message

ChatCompletionMessage(content='The model would give you a "Good morning" greeting if it is 23 o\'clock.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)